In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [3]:
import pandas as pd
realDF = pd.read_csv('../data-processing/cresci-2017/genuine_accounts_processed.csv')
ss1DF = pd.read_csv('../data-processing/cresci-2017/social_spambots_1_processed.csv')
ss2DF = pd.read_csv('../data-processing/cresci-2017/social_spambots_2_processed.csv')
ss3DF = pd.read_csv('../data-processing/cresci-2017/social_spambots_3_processed.csv')
ffDF = pd.read_csv('../data-processing/cresci-2017/fake_followers_processed.csv')
ts1DF = pd.read_csv('../data-processing/cresci-2017/traditional_spambots_1_processed.csv')
coaDF = pd.read_csv('../data-processing/college-official-accounts/coa_processed.csv')


In [6]:
# removing some entries from data
del coaDF['url_youtube']
del coaDF['favorite'], realDF['favorite'], ss1DF['favorite'], ss2DF['favorite'], ss3DF['favorite'], ffDF['favorite'], ts1DF['favorite']
del coaDF['favorite_over_friend'], realDF['favorite_over_friend'], ss1DF['favorite_over_friend'], ss2DF['favorite_over_friend'], ss3DF['favorite_over_friend'], ffDF['favorite_over_friend'], ts1DF['favorite_over_friend']

In [7]:
def data_split (df, split=0.7):
    train_num = int(len(df.index)*split)
    return df.iloc[:train_num,:], df.iloc[train_num+1:,:]

realTrain, realTest = data_split(realDF)
ss1Train, ss1Test = data_split(ss1DF)
ss2Train, ss2Test = data_split(ss2DF)
ss3Train, ss3Test = data_split(ss3DF)
ts1Train, ts1Test = data_split(ts1DF)
ffTrain, ffTest = data_split(ffDF)
coaTrain, coaTest = data_split(coaDF)

In [11]:
print(coaTrain.shape, realTrain.shape)

(2499, 49) (2431, 49)


In [12]:
def trainRF (reals, bots, n_estimators=100):

    data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
    target = np.array([0] * sum(len(real.index) for real in reals) + [1] * sum(len(bot.index) for bot in bots))
    rfc = RandomForestClassifier(n_estimators=n_estimators)
    rfc = rfc.fit(data, target)
    return rfc

In [13]:
def testRF(reals, bots, RF):
    data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
    target = np.array([0] * sum(len(real.index) for real in reals) + [1] * sum(len(bot.index) for bot in bots))
    return RF.score(data, target)

In [14]:
def testRF_ESC(reals, bots, RFs):
    data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
    target = np.array([0] * sum(len(real.index) for real in reals) + [1] * sum(len(bot.index) for bot in bots))
    totalAccConf = 0.0
    totalAccNum = 0
    for i in range(data.shape[0]):
        botMax = max(float(RF.predict_proba([data[i]])[0][1]) for RF in RFs)
        genuineMax = min(float(RF.predict_proba([data[i]])[0][1]) for RF in RFs)
        if botMax >= 0.5:
            totalAccConf += abs(botMax - float(target[i]))
            if abs(botMax - float(target[i])) >= 0.5:
                print("Error Pred 1, ", i, data[i], target[i], [RF.predict_proba([data[i]])[0] for RF in RFs], botMax)
            else:
                totalAccNum += 1
        elif genuineMax <= 0.5:
            totalAccConf += abs(float(target[i]) - genuineMax)
            if abs(float(target[i]) - genuineMax) >= 0.5:
                print("Error Pred 2, ", i, data[i], target[i], [RF.predict_proba([data[i]])[0] for RF in RFs], genuineMax)
            else:
                totalAccNum += 1
    return float(totalAccNum)/data.shape[0], totalAccConf/data.shape[0]

In [15]:
RF = []
RF.append(trainRF([realTrain], [ss1Train, ss2Train, ss3Train, ffTrain, ts1Train, coaTrain]))
RF.append(trainRF([realTrain], [ss1Train, ss2Train, ss3Train]))
RF.append(trainRF([realTrain], [ts1Train]))
RF.append(trainRF([realTrain], [coaTrain]))
RF.append(trainRF([realTrain], [ffTrain]))

/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/2087584835.py:3: FutureWarning: In a future version of pandas all argum

In [17]:
print(testRF([realTest], [ss1Test, ss2Test, ss3Test, ffTest, ts1Test, coaTrain], RF[0]))

0.9924026590693258


/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/1317824033.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()


In [18]:
correctRate, correctConf = testRF_ESC([realTest], [coaTest], RF)

/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/1946100403.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()


Error Pred 1,  49 [  4.    4.   40.    0.    0.1 -36.    0.1   0.    0.    0.    0.    0.
   1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    2.    1.    0.    0.4   0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. ] 0 [array([0.5, 0.5]), array([0.99, 0.01]), array([0.86, 0.14]), array([0.94, 0.06]), array([0.63, 0.37])] 0.5
Error Pred 1,  95 [ 5.20000000e+01  2.60000000e+01  6.20000000e+01  0.00000000e+00
  4.19354839e-01 -3.60000000e+01  8.38709677e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  8.00000000e-01  1.15384615e+00  6.92646138e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.84615385

In [19]:
print(correctRate, correctConf)

0.9919545669663985 0.0067439659252247946


In [27]:
singleRF = trainRF([coaTrain], [ss1Train, ss2Train, ss3Train, ts1Train])
print(testRF([coaTest], [ss1Test, ss2Test, ss3Test, ts1Test], singleRF))

/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()


1.0


/var/folders/qx/bly71f215mx_406j1_my1ghh0000gp/T/ipykernel_23104/1317824033.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
